In [1]:
URL="http://10.128.0.5:8080"

In [2]:
!trino $URL --execute "show catalogs"

Query 20231206_004711_00000_m2a9b failed: Trino server is still initializing



In [5]:
!trino $URL --execute "show schemas from mysql"

"college"
"final_project"
"information_schema"
"performance_schema"
"sakila"


In [28]:
!trino $URL --execute "show tables from mysql.college"

"class"
"instructor"
"student"
"teaches"


In [24]:
!trino $URL --execute "select * from mysql.college.class limit 3"

"aprilz      ","CS373 ","Software Engineering","3","  "
"aprilz      ","M358K ","Applied Statistics","3","  "
"bzen26      ","CS313E","Elements of Software Engineering","3","B+"


In [33]:
!trino $URL --execute "show schemas from postgres"

"college"
"final_project"
"information_schema"
"pagila"
"pg_catalog"
"public"
"shopify"


In [10]:
!trino $URL --execute "show tables from postgres.college"

"class"
"instructor"
"student"
"takes"
"teaches"


In [11]:
!trino $URL --execute "select * from postgres.college.class limit 3"

"CS327E","Elements of Databases","3"
"CS347 ","Data Management","3"
"CS330E","Elements of Software Engineering","3"


In [12]:
!trino $URL --execute "show schemas from mongo"

"college"
"final_project"
"information_schema"


In [13]:
!trino $URL --execute "show tables from mongo.college"

"takes"


In [14]:
!trino $URL --execute "select * from mongo.college.takes limit 3"

"kev18","CS329E"
"bzen26","CS329E"
"jc","CS327E"


In [15]:
!trino $URL --execute "show schemas from bigquery"

"college"
"final_project"
"information_schema"
"school_enrollments"
"test2"


In [16]:
!trino $URL --execute "show tables from bigquery.college"

"class"
"instructor"
"instructor2"
"student"
"student_copy"
"takes"
"teaches"
"v_enrollments"


In [17]:
!trino $URL --execute "select * from bigquery.college.instructor limit 3"

"neeman","Joe Neeman","Mathematics"
"koch","Hans Koch","Mathematics"
"mueller","Peter Mueller","Mathematics"


We changed the student table to class becauase we changed the student table a while back and it caused this error now.

In [25]:
!trino $URL --execute "select * from mysql.college.class s \
join mongo.college.takes ta on s.sid = ta.sid \
join postgres.college.class c on ta.cno = c.cno \
join postgres.college.teaches te on c.cno = te.cno \
join bigquery.college.instructor i on te.tid = i.tid \
order by s.sid \
limit 3"

"bzen26      ","CS313E","Elements of Software Engineering","3","B+","bzen26","CS329E","CS329E","Elements of Web Programming","3","mitra","CS329E","mitra","Shyamal Mitra","Computer Science"
"bzen26      ","CS313E","Elements of Software Engineering","3","B+","bzen26","CS329E","CS329E","Elements of Web Programming","3","mitra","CS329E","mitra","Shyamal Mitra","Computer Science"
"jc          ","CS331E","Elements of Software Engineering","3","  ","jc","CS327E","CS327E","Elements of Databases","3","scohen","CS327E","scohen","Shirley Cohen","Computer Science"


# Part 2

## Question 1

Who are the shoppers who have reservations in a different city from their own city and who paid for their reservations in US dollars? For those shoppers, return their cust_id, email, along with their reservation count. Order the results by cust_id. Limit the results to 5 records.

In [45]:
!trino $URL --execute "select p.cust_id, p.email, count(*) from mysql.final_project.person p \
join mongo.final_project.ticketing t on p.cust_id = t.cust_id \
join postgres.final_project.reservations r on p.cust_id = r.cust_id \
join bigquery.final_project.currency ct on ct.curr_code = t.curr_code \
join bigquery.final_project.currency cr on cr.cnty_code = r.cnt_code \
where p.city <> r.city and ct.curr_code='USD'\
group by p.cust_id, p.email \
order by p.cust_id \
limit 5"

In [ ]:
We have no output because none of our data fits this query. 

In [47]:
!trino $URL --execute "select p.cust_id, p.email, count(*) from mysql.final_project.person p \
join mongo.final_project.ticketing t on p.cust_id = t.cust_id \
join postgres.final_project.reservations r on p.cust_id = r.cust_id \
join bigquery.final_project.currency ct on ct.curr_code = t.curr_code \
join bigquery.final_project.currency cr on cr.cnty_code = r.cnt_code \
where p.city <> r.city \
group by p.cust_id, p.email \
order by p.cust_id \
limit 5"

"4","mejiasamantha@example.net","6"
"6","kevinanderson@example.net","10"
"11","johnsonangela@example.org","12"
"19","spotter@example.com","3"
"24","angelagonzales@example.com","8"


## Question 2

Who are the shoppers who have bought non-stop tickets on American Airlines with a
departure date between 01/01/2020 and 12/31/2024? Return the shopper’s cust_id, email,
dep_airport and dep_date. Order the results by dep_date. Limit the results to 5 records.

In [88]:
!trino $URL --execute "select p.cust_id, p.email, t.dep_airport, t.dep_date from mysql.final_project.person p \
join mongo.final_project.ticketing t on p.cust_id = t.cust_id \
where t.stops=0 AND t.dep_date BETWEEN '2020-01-01' AND '2024-12-31' \
order by t.dep_date \
limit 5"

"2","jstrong@example.org","Shimojishima airport","2020-03-13"
"35","mjohnson@example.net","Fort Lauderdale Hollywood International airport","2020-03-13"
"38","thomas00@example.net","Yelovaya airport","2020-06-16"
"4","mejiasamantha@example.net","Baltimore-Washington International Thurgood Mars","2020-09-12"
"7","traceystewart@example.org","Gimpo International airport","2020-09-12"


## Question 3

Which shoppers have reservations and tickets with matching dates such that the arr_date
and dep_date are equal between reservations and ticketing? For such shoppers, sum up their
pmt_amt and tik_amt and return this sum as trip_amount. Also, return the shoppers’ cust_id,
email, pmt_amt, tik_amt. Order the results by trip_amount in descending order. Limit the results
to 5 records.


In [79]:
!trino $URL --execute "select p.cust_id, p.email, sum(cast(r.pmt_amt as decimal)) as pmt_amt, sum(cast(t.tik_amt as int)) as tik_amt, sum(cast(r.pmt_amt as decimal)+tik_amt) as trip_amount from mysql.final_project.person p \
join mongo.final_project.ticketing t on p.cust_id = t.cust_id \
join postgres.final_project.reservations r on p.cust_id = r.cust_id \
join bigquery.final_project.currency ct on ct.curr_code = t.curr_code \
join bigquery.final_project.currency cr on cr.cnty_code = r.cnt_code \
where t.dep_date>=r.dep_date and t.arr_date<=r.arr_date \
group by p.cust_id, p.email \
order by trip_amount desc \
limit 5"

"44","costachristopher@example.com","2262","15620","17882"
"26","kelly33@example.com","1472","6336","7808"
"5","ljames@example.com","178","758","936"
"10","michael82@example.com","252","480","732"
"45","davidsonemily@example.net","225","45","270"


# PART 3

### Adding stops as a column in the ticketing table (mongo) (DDL and DML) (create copy of ticketing table with all its data and extra stops column and then deletes old one and renames new one)

In [82]:
!trino $URL --execute "create table mongo.final_project.ticket as select *, cast(floor(rand()*3) as int) as stops from mongo.final_project.ticketing"

CREATE TABLE: 401 rows


In [86]:
!trino $URL --execute "drop table mongo.final_project.ticketing;"

DROP TABLE


In [87]:
!trino $URL --execute "alter table mongo.final_project.ticket rename to mongo.final_project.ticketing"

RENAME TABLE


### Adding fav_food as a column in the ticketing table (mysql) (DDL)

In [30]:
!trino $URL --execute "ALTER TABLE mysql.final_project.person add column fav_food VARCHAR(50);"

ADD COLUMN


### Updating everyone's fav_food to be Spaghetti and Meatballs (DML)

In [35]:
!trino $URL --execute "UPDATE mysql.final_project.person SET fav_food = 'Spaghetti and Meatballs';"

UPDATE: 50 rows


### Select everyone from TX who has a favorite food of Spaghetti and Meatballs (Select)

In [38]:
!trino $URL --execute "select p.first_name, p.last_name from mysql.final_project.person p \
where p.fav_food = 'Spaghetti and Meatballs' and p.state = 'TX'"

"Tina","Brown"


### Add column stars (postgres) (DDL)

In [50]:
!trino $URL --execute "ALTER TABLE postgres.final_project.reservations add column stars INT;"

ADD COLUMN


### update stars to be 4 for all (DML)

In [53]:
!trino $URL --execute "UPDATE postgres.final_project.reservations SET stars = 4;"

UPDATE: 100 rows


### find all reservations where the stars is greater than 3 and the state is Texas. (select)

In [56]:
!trino $URL --execute "select r.prp_nm, r.prp_ch from postgres.final_project.reservations r where r.stars > 3 and state = 'TX' "

"Snyder and Sons","Lane-Romero LLC"
"Parker-Morales","Leon-Sullivan LLC"


### Adding exchange_rate_to_USD as a column in the ticketing table (bigquery) (DDL and DML) (same method as mongo, but for some reason bigquery does not like renaming tables, so had to copy the new table with the old name and drop the new table)

In [42]:
!trino $URL --execute "create table bigquery.final_project.curr as select *, (rand()*3) as exchange_rate_to_USD from bigquery.final_project.currency"

CREATE TABLE: 120 rows


In [43]:
!trino $URL --execute "drop table bigquery.final_project.currency;"

DROP TABLE


In [46]:
!trino $URL --execute "create table bigquery.final_project.currency as select * from bigquery.final_project.curr"

CREATE TABLE: 120 rows


In [47]:
!trino $URL --execute "drop table bigquery.final_project.curr;"

DROP TABLE


### select all currencies that have an exchange rate over 2 (select)

In [48]:
!trino $URL --execute "select curr_name, exchange_rate_to_USD from bigquery.final_project.currency where exchange_rate_to_USD >2 limit 5"

"Armenian_dram","2.5432259065853313"
"Azerbaijani_manat","2.687999794141315"
"Brunei_dollar","2.231766536360813"
"Bolivian_boliviano","2.2176345817300946"
"Brazilian_real","2.317745617380419"


### select statement that looks for a person that lives in delaware and made a reservation using JEP, or Jersey Pound and returns cust_id, cust_state, state of reservation, the pmt_amt, the number of stops in flight, and the currency used

In [114]:
!trino $URL --execute "select p.cust_id, p.state, r.state, r.pmt_amt, t.stops, cr.curr_name from mysql.final_project.person p \
join mongo.final_project.ticketing t on p.cust_id = t.cust_id \
join postgres.final_project.reservations r on p.cust_id = r.cust_id \
join bigquery.final_project.currency cr on cr.cnty_code = r.cnt_code \
where p.state='DE' and cr.curr_code='JEP' \
order by p.cust_id \
limit 5"

"28","DE","FM","52.97","2","Jersey_pound"
"28","DE","FM","52.97","1","Jersey_pound"
"28","DE","FM","52.97","1","Jersey_pound"
"28","DE","FM","52.97","1","Jersey_pound"
"28","DE","FM","52.97","1","Jersey_pound"


### select statement that looks for a person that spent below average in a reservation and above average in the ticket (subquery)

In [111]:
!trino $URL --execute "select p.cust_id, p.first_name, p.last_name, r.pmt_amt, t.tik_amt from mysql.final_project.person p \
join postgres.final_project.reservations r on p.cust_id = r.cust_id \
join mongo.final_project.ticketing t on p.cust_id = t.cust_id \
where cast(r.pmt_amt as decimal)<(select avg(cast(pmt_amt as decimal)) from postgres.final_project.reservations)\
and cast(t.tik_amt as decimal)>(select avg(cast(tik_amt as decimal)) from mongo.final_project.ticketing) \
group by p.cust_id, p.first_name, p.last_name, r. pmt_amt, t.tik_amt\
order by p.cust_id \
limit 20"

"2","Kathryn","Johnson","15.12","427"
"2","Kathryn","Johnson","19.23","357"
"2","Kathryn","Johnson","15.12","398"
"2","Kathryn","Johnson","15.12","474"
"2","Kathryn","Johnson","19.23","427"
"2","Kathryn","Johnson","19.23","474"
"2","Kathryn","Johnson","19.23","398"
"2","Kathryn","Johnson","19.23","449"
"2","Kathryn","Johnson","15.12","357"
"2","Kathryn","Johnson","15.12","449"
"3","Cheryl","Poole","19.94","223"
"3","Cheryl","Poole","19.94","346"
"3","Cheryl","Poole","19.94","405"
"5","Zachary","Galvan","15.48","235"
"5","Zachary","Galvan","15.48","340"
"5","Zachary","Galvan","15.48","264"
"5","Zachary","Galvan","15.48","328"
"5","Zachary","Galvan","15.48","379"
"8","William","Thompson","45.18","412"
"8","William","Thompson","48.41","412"


Initially we were doing question 2 and realized we had no stops column in ticketing because we forgot to add it. Instead of adding it in mongo, we decided to demonstrate trino's DDL and DML and select ability. We could not use alter table for some reason, so we had to use a different method. We created table ticket which was a copy of ticketing with the stops column added to it. Then we dropped ticketing and renamed ticket to ticketing. 
First we ran mySQL DDL, DML, and select statements. creating, altering, and modyfying the table against mysql to make a favorite food column. All of the operations were allowed under trinos functionality. Then we selected everyone who was in Texas that had a favorite food of Spaghetti and Meatballs. Then we created against Postgres, and it was also under trino's finctionality, but when we tried to make a stars column have a random number from 1-5, trino did not support such complex operations. We selected reservations at restaurants who were in texas and above 3 stars. 
For bigquery, we tried to add a column into the currency table called exchange_rate_to_USD. However, we ran into the same problem as mongo, where for some reason we could not simply use the alter command to add a column to the table. We instead had to make a new table called curr with all of the content of the old one plus the column we wanted. Then we had to delete the old table and rename the new one. However, unlike mongo, we couldnt simply rename the table for some reason, so we had to create an exact copy of the new table with the name currency, and then delete table curr. There is a simple select statement that looks at all exchange rates bigger than 2 and returns it limited by 5 outpus.
For the next call, we wanted to do a select statement that returned data from the every table. The select statement returns customer id, the state they live in, the state the reservation is in, the payment amount of the reservation, the number of stops in the flight, and the name of the currency they paid with. This not only tested how easy it is to access all the databases at once, it also checked to see if we properly inserted stops into the ticket table. 
Finally, we did one more select statement that uses nested queries and aggregate functions like average. We returned the persons customer id, first name, last name, the reservation payment amount, and the ticket amount, and filtered using a where statement, where the reservation payment amount had to be lower than the average payment amount, which we found using a nested select statement with the avg function, and the ticket payment had to be higher than the average using the same method. 
We realized that excpet for mysql, it was hard to alter any tables that inside of trino. We could insert, update, create, and delete pretty easily, but whenever we tried to alter a table, it would output an error. Therefore, while Trino is a good tool to use all of the databases at once, there a few flaws when using it.